In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-08 03:36:18--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-02-08 03:36:18 (6.59 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read the review file and convert to DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id',"star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1QX6706ZWJ1P5|          5|            0|          0|   N|                Y|
|R3QWMLJHIW6P37|          5|            0|          0|   N|                Y|
|R14Z1VR1N0Z9G6|          5|            1|          1|   N|                Y|
| R25ZRJL0GH0U0|          2|            0|          0|   N|                Y|
|R3837KYH7AZNIY|          4|            0|          1|   N|                Y|
|R2MN0QYCY6EVIV|          5|            0|          0|   N|                Y|
|R3AN2UJ1D42ZS0|          5|            0|          0|   N|                Y|
|R3U29ZLUWEEK4M|          5|            0|          0|   N|                Y|
|R16ZDMJJHK796C|          5|            0|          0|   N|                N|
| RRRDOEJZD1Y22|          4|            0|          0|   N|     

In [6]:
# 1. Create a new DataFrame where total_votes count is equal to or greater than 20 
vote_count_df = vine_df.filter(vine_df.total_votes >= 20)
vote_count_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2PT2X8FMYFCV3|          5|           39|         41|   N|                Y|
| RGCUQSH80SYY7|          1|           36|         47|   N|                N|
|R3E6EI073KP0AL|          2|           21|         36|   N|                Y|
|R3MYTXW7B4Z6YW|          4|          687|        711|   N|                Y|
|R2WMF1IWN5ZYO4|          5|           18|         24|   N|                Y|
| ROX6L7VJFS2P7|          5|          339|        371|   N|                N|
| RRKAST7D4QTW8|          1|           20|         27|   N|                N|
|R2QBKTWBDPL8BF|          5|           65|         83|   N|                N|
|R3ULT9I0U79N8L|          5|           49|         57|   N|                N|
|R20LABVEB8O83Q|          3|           11|         23|   N|     

In [7]:
# 2. Create a DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
voting_percent_df = vote_count_df.filter((vote_count_df.helpful_votes/vote_count_df.total_votes)>= 0.5)
voting_percent_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2PT2X8FMYFCV3|          5|           39|         41|   N|                Y|
| RGCUQSH80SYY7|          1|           36|         47|   N|                N|
|R3E6EI073KP0AL|          2|           21|         36|   N|                Y|
|R3MYTXW7B4Z6YW|          4|          687|        711|   N|                Y|
|R2WMF1IWN5ZYO4|          5|           18|         24|   N|                Y|
| ROX6L7VJFS2P7|          5|          339|        371|   N|                N|
| RRKAST7D4QTW8|          1|           20|         27|   N|                N|
|R2QBKTWBDPL8BF|          5|           65|         83|   N|                N|
|R3ULT9I0U79N8L|          5|           49|         57|   N|                N|
| RAC19YVDYSIKB|          5|          161|        178|   N|     

In [8]:
# 3. Create a DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid)
paid_reviews_df = voting_percent_df.filter(voting_percent_df.vine == "Y")
paid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3R2RHQWEKX5NL|          5|           81|         89|   Y|                N|
|R1RZ4JZO8DAP7H|          3|          200|        227|   Y|                N|
| RQ94HI0WM8KIG|          5|          174|        188|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+



In [14]:
# 4. Create a DataFrame that retrieves all the rows where a review was written as part of the Vine program (unpaid)

unpaid_reviews_df = voting_percent_df.filter(voting_percent_df.vine == "N")
unpaid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2PT2X8FMYFCV3|          5|           39|         41|   N|                Y|
| RGCUQSH80SYY7|          1|           36|         47|   N|                N|
|R3E6EI073KP0AL|          2|           21|         36|   N|                Y|
|R3MYTXW7B4Z6YW|          4|          687|        711|   N|                Y|
|R2WMF1IWN5ZYO4|          5|           18|         24|   N|                Y|
| ROX6L7VJFS2P7|          5|          339|        371|   N|                N|
| RRKAST7D4QTW8|          1|           20|         27|   N|                N|
|R2QBKTWBDPL8BF|          5|           65|         83|   N|                N|
|R3ULT9I0U79N8L|          5|           49|         57|   N|                N|
| RAC19YVDYSIKB|          5|          161|        178|   N|     

In [9]:
paid_reviews_count = paid_reviews_df.count()
paid_reviews_count

3

In [10]:
five_star_paid_reviews_count = paid_reviews_df.filter(paid_reviews_df.star_rating == '5').count()
five_star_paid_reviews_count

2

In [11]:
five_star_paid_reviews_percent_count = (five_star_paid_reviews_count/paid_reviews_count)*100
five_star_paid_reviews_percent_count

66.66666666666666

In [15]:
unpaid_reviews_count= unpaid_reviews_df.count()
unpaid_reviews_count

3094

In [16]:
five_star_unpaid_reviews_count = unpaid_reviews_df.filter(unpaid_reviews_df.star_rating == '5').count()
five_star_unpaid_reviews_count

1704

In [17]:
five_star_unpaid_reviews_percent_count = (five_star_unpaid_reviews_count/unpaid_reviews_count)*100
five_star_unpaid_reviews_percent_count

55.07433742727861